In [1]:
#Import Modules
import pandas as pd

In [2]:
#Path the file locations of the csv files
gdp = "data/original/gpd_states.csv"
unem = "data/original/Unemployment_States_Annual.csv"

#Convert csv files into dataframes
gdp_df = pd.read_csv(gdp)
unem_df = pd.read_csv(unem, index_col =0)

In [3]:
#Drop the unwanted row
drop_gdp = gdp_df.drop(0)
drop_gdp.head()
drop_gdp = drop_gdp.set_index(["GeoName"])

In [4]:
#Get the states into a list for future referance
state_list = [ "Alabama", "Arkansas", 
            "Arizona", "California", "Colorado", "Connecticut", 
            "District of Columbia", "Delaware", "Florida", "Georgia", 
            "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", 
            "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", 
            "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", 
            "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", 
            "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", 
            "Rhode Island", "South Carolina", "South Dakota", "Tennessee", 
            "Texas", "Utah", "Virginia", "Vermont", "Washington", 
            "Wisconsin", "West Virginia", "Wyoming"]

In [5]:
#Locate the first state in our data
gdp_data = pd.DataFrame(drop_gdp.loc["Alaska",:])

#container for loop appending
state_cont = []

#Loop to get repeating state name in column and setting up new loop
for x in range(len(gdp_data)):
    state_cont.append(gdp_data.columns[0])
gdp_data["GeoName"] = state_cont
gdp_data = gdp_data.set_index(["GeoName",gdp_data.index]) 
gdp_data.columns=["GDP"]

#Iterate through all states to get data in a preferred format for mongodb
for state in state_list:

    gdp_new = pd.DataFrame(drop_gdp.loc[state,:])

    state_cont = []

    for x in range(len(gdp_new)):
        state_cont.append(gdp_new.columns[0])
    gdp_new["GeoName"] = state_cont
    gdp_new = gdp_new.set_index(["GeoName",gdp_new.index])
    gdp_new.columns=["GDP"]
    gdp_data = gdp_data.append(gdp_new)
gdp_data   

GDP
GeoName              
Alaska  2011  56896.3
        2012  58283.6
        2013  57247.7
        2014  56484.9
        2015  51490.9
...               ...
Wyoming 2017  36863.7
        2018  39032.3
        2019  39600.8
        2020  36323.5
        2021  41622.4

[561 rows x 1 columns]

In [6]:
#Locate the first state in our data
unem_data = pd.DataFrame(unem_df.T.loc["Alaska",:])

#container for loop appending
state_cont = []

#Loop to get repeating state name in column and setting up new loop
for x in range(len(unem_data)):
    state_cont.append(unem_data.columns[0])
unem_data["State"] = state_cont
unem_data = unem_data.set_index(["State",unem_data.index]) 
unem_data.columns=["Unemployment Rate"]

#Iterate through all states to get data in a preferred format for mongodb
for state in state_list:

    unem_new = pd.DataFrame(unem_df.T.loc[state,:])

    state_cont = []

    for x in range(len(unem_new)):
        state_cont.append(unem_new.columns[0])
    unem_new["State"] = state_cont
    unem_new = unem_new.set_index(["State",unem_new.index])
    unem_new.columns=["Unemployment Rate"]
    unem_data = unem_data.append(unem_new)
unem_data  

Unemployment Rate
State   DATE                   
Alaska  2011                7.8
        2012                7.2
        2013                7.0
        2014                6.7
        2015                6.3
...                         ...
Wyoming 2017                4.3
        2018                4.1
        2019                3.7
        2020                5.8
        2021                4.5

[561 rows x 1 columns]

In [7]:
#Rename index header and give empty header a name
gdp_data.index.names = ['State', 'DATE']
gdp_data

GDP
State   DATE         
Alaska  2011  56896.3
        2012  58283.6
        2013  57247.7
        2014  56484.9
        2015  51490.9
...               ...
Wyoming 2017  36863.7
        2018  39032.3
        2019  39600.8
        2020  36323.5
        2021  41622.4

[561 rows x 1 columns]

In [8]:
level=1
unem_data.index = unem_data.index.set_levels(unem_data.index.levels[level].astype(str),level=level)


In [9]:
unem_data.index.dtypes

State    object
DATE     object
dtype: object

In [10]:
combo_df = pd.merge(unem_data, gdp_data, how = "outer", on = ["State","DATE"])
combo_df.to_csv("data/unem_gdp.csv")

In [11]:
combo_df.index.dtypes

State    object
DATE     object
dtype: object

In [12]:
edu = "data/education.csv"

#Convert csv files into dataframes
edu_df = pd.read_csv(edu)

edu_df = edu_df.set_index(["State","DATE"])
level=1
edu_df.index = edu_df.index.set_levels(edu_df.index.levels[level].astype(str),level=level)

In [33]:
final_df = pd.merge(edu_df, combo_df, how = "outer", on = ["State","DATE"])
final_df['GDP'] = final_df['GDP']/10000
final_df = final_df.dropna()
final_df['Employment Rate'] = 100 - final_df['Unemployment Rate']
final_df

College  High School  Unemployment Rate      GDP  \
State   DATE                                                     
Alaska  2011     26.4         91.8                7.8  5.68963   
        2012     28.0         92.0                7.2  5.82836   
        2013     28.0         91.6                7.0  5.72477   
        2014     28.0         92.9                6.7  5.64849   
        2015     29.7         92.6                6.3  5.14909   
...               ...          ...                ...      ...   
Wyoming 2016     27.1         93.2                5.4  3.58790   
        2017     27.6         92.9                4.3  3.68637   
        2018     26.9         93.3                4.1  3.90323   
        2019     29.1         94.5                3.7  3.96008   
        2020     28.2         93.8                5.8  3.63235   

              Employment Rate  
State   DATE                   
Alaska  2011             92.2  
        2012             92.8  
        2013             93.0  
        2014             93.3  
        2015             93.7  
...                       ...  
Wyoming 2016             94.6  
        2017             95.7  
        2018             95.9  
        2019             96.3  
        2020             94.2  

[510 rows x 5 columns]

In [34]:
final_df.to_csv("data/happiness.csv")